Import packages

In [73]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale, StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import rcParams

from keras.layers import Input, Dense, Lambda, concatenate
from keras.models import Model, load_model, Sequential, model_from_json
from keras import backend as K
from keras import metrics, optimizers, regularizers
from keras.objectives import categorical_crossentropy, mean_squared_error
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

# fix random seed for reproducibility
tf.set_random_seed(123)
np.random.seed(123)

Load data

In [74]:
# load Concrete_Data.xls dataset, X: features, contains concrete mixes formula 
df_formula = pd.read_excel('Concrete_Data.xls')
df_epd = pd.read_excel('environmental_impact.xlsx')
X = df_formula.values[:,:7]

Samples separated into 6 groups <br>
Group 0: < 3d <br>
Group 1: 7d <br>
Group 2: 14d <br>
Group 3: 28d <br>
Group 4: 56d <br>
Group 5: > 90d

In [75]:
day_to_idx = {1:0,
              3:0,
              7:1,
             14:2,
             28:3,
             56:4,
             90:5,
             91:5,
             100:5,
             120:5,
             180:5,
             270:5,
             360:5,
             365:5}

Convert indices to one hot

In [76]:
indices = []

day_raw = df_formula.values[:,7]

for i, day in enumerate(day_raw):
    indices.append(day_to_idx[day])
indices = np.array(indices)
one_hot_vecs = np.zeros((day_raw.shape[0], 6)) 
one_hot_vecs[np.arange(day_raw.shape[0]), np.array(indices)] = 1

Get indices of samples from different groups, for later usage

In [77]:
group_0_train = np.where(indices == 0)[0]
group_1_train = np.where(indices == 1)[0]
group_2_train = np.where(indices == 2)[0]
group_3_train = np.where(indices == 3)[0]
group_4_train = np.where(indices == 4)[0]
group_5_train = np.where(indices == 5)[0]

Stack up group info, strength, and environmental impact

In [78]:
Y = np.hstack((one_hot_vecs, df_formula.values[:,8].reshape(-1,1), df_epd.values[:,:12]))

Setup scaler for X and Y

In [79]:
scaler_X = MinMaxScaler().fit(X)
scaler_Y = MinMaxScaler().fit(Y)
X = scaler_X.transform(X)  
Y = scaler_Y.transform(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Define input tensor

In [80]:
n_x = X.shape[1]
n_y = Y.shape[1]
n_z = 2

# Q(z|X,y) -- encoder
formula = Input(shape=(n_x,))
cond = Input(shape=(n_y,))

Initialize encoder layers

In [81]:
inputs = concatenate([formula, cond])
enc_hidden_1 = Dense(30, activation='relu')(inputs)
enc_hidden_2 = Dense(25, activation='relu')(enc_hidden_1)
z_mean = Dense(n_z)(enc_hidden_2)
z_log_var = Dense(n_z)(enc_hidden_2)

Define sampling function

In [82]:
def sample_z(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], n_z), mean=0., stddev=1.0)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [83]:
# Sample z ~ Q(z|X,y)
z = Lambda(sample_z)([z_mean, z_log_var])
z_cond = concatenate([z, cond])

In [84]:
# P(X|z,y) -- decoder
dec_layer_1 = Dense(25, activation='relu') 
dec_layer_2 = Dense(30, activation='relu') 
dec_out = Dense(n_x, activation='sigmoid') #, activation='sigmoid'

dec_hidden_1 = dec_layer_1(z_cond)
dec_hidden_2 = dec_layer_2(dec_hidden_1)
reconstructed = dec_out(dec_hidden_2)

In [85]:
# end-to-end autoencoder
cvae = Model([formula, cond], reconstructed)

# encoder, from inputs to latent space
encoder = Model([formula, cond], z)

In [86]:
z_for_gen = Input(shape=(n_z,))
z_cond_for_gen = concatenate([z_for_gen, cond])
dec_hidden_1_for_gen = dec_layer_1(z_cond_for_gen)
dec_hidden_2_for_gen = dec_layer_2(dec_hidden_1_for_gen)
reconstructed_for_gen = dec_out(dec_hidden_2_for_gen)

# generator, from latent space to reconstructed inputs
generator = Model([z_for_gen, cond], reconstructed_for_gen)

Define loss

In [87]:
def cvae_loss(feature, reconstructed):
    reconstruction_loss = mean_squared_error(feature, reconstructed)
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return reconstruction_loss + kl_loss

Compile model

In [88]:
adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
cvae.compile(optimizer=adam, loss=cvae_loss)
cvae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 7)            0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 19)           0                                            
__________________________________________________________________________________________________
concatenate_19 (Concatenate)    (None, 26)           0           input_16[0][0]                   
                                                                 input_17[0][0]                   
__________________________________________________________________________________________________
dense_45 (Dense)                (None, 30)           810         concatenate_19[0][0]             
__________

In [89]:
n_epoch = 100
n_batch = 10

Train model

In [90]:
checkpointer = ModelCheckpoint(filepath = "model_autoencoder.h5",
                               verbose = 0,
                               save_best_only = True)

tensorboard = TensorBoard(log_dir = './logs',
                          histogram_freq = 0,
                          write_graph = True,
                          write_images = True)

history = cvae.fit([X_train, Y_train], X_train,
                epochs=n_epoch,
                batch_size=n_batch,
                shuffle=True,
                callbacks = [EarlyStopping(patience = 5)],
                validation_data=([X_test, Y_test], X_test))

Train on 824 samples, validate on 206 samples
Epoch 1/100
824/824 [==============================] - 2s 2ms/step - loss: 0.0751 - val_loss: 0.0508
Epoch 2/100
824/824 [==============================] - 0s 322us/step - loss: 0.0505 - val_loss: 0.0451
Epoch 3/100
824/824 [==============================] - 0s 544us/step - loss: 0.0457 - val_loss: 0.0417
Epoch 4/100
824/824 [==============================] - 0s 523us/step - loss: 0.0423 - val_loss: 0.0384
Epoch 5/100
824/824 [==============================] - 0s 346us/step - loss: 0.0398 - val_loss: 0.0366
Epoch 6/100
824/824 [==============================] - 0s 445us/step - loss: 0.0375 - val_loss: 0.0344
Epoch 7/100
824/824 [==============================] - 0s 278us/step - loss: 0.0358 - val_loss: 0.0321
Epoch 8/100
824/824 [==============================] - 0s 491us/step - loss: 0.0333 - val_loss: 0.0298
Epoch 9/100
824/824 [==============================] - 0s 479us/step - loss: 0.0319 - val_loss: 0.0286
Epoch 10/100
824/824 [=======

Generate new samples

In [91]:
num_of_samples = 60000
day_vecs = np.zeros((num_of_samples, 6))
day_idx = np.random.choice(6, num_of_samples)
group_0 = np.where(day_idx == 0)[0]
group_1 = np.where(day_idx == 1)[0]
group_2 = np.where(day_idx == 2)[0]
group_3 = np.where(day_idx == 3)[0]
group_4 = np.where(day_idx == 4)[0]
group_5 = np.where(day_idx == 5)[0]
day_vecs = np.zeros((num_of_samples, 6))
day_vecs[np.arange(num_of_samples), day_idx] = 1
np.random.seed(1)
strength_and_environmental = np.random.uniform(0, 1, (num_of_samples, 13))
mean = [0,0]
cov = [[1, 0], [0, 1]]
np.random.seed(1)
Z_sampling = np.random.multivariate_normal(mean, cov, num_of_samples)
Y_sampling = np.hstack((day_vecs, strength_and_environmental))
samples_scaled = generator.predict([Z_sampling, Y_sampling])
generated_samples = scaler_X.inverse_transform(samples_scaled)